In [102]:
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromeium-chromedriver > /dev/null 2>&1

지정된 경로를 찾을 수 없습니다.
지정된 경로를 찾을 수 없습니다.
지정된 경로를 찾을 수 없습니다.


In [103]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [106]:
options=webdriver.ChromeOptions()
options.add.argument('--headless')
options.add.argument('--no-sandbox')
options.add.argument('--single-process')
options.add.argument('--disable-dev-shm-usage')
driver=webdriver.Chrome('chromedriver', options=options)

AttributeError: 'Options' object has no attribute 'add'

In [139]:
url='https://www.danawa.com/?src=adwords&kw=GA0000020&gclid=CjwKCAiA5Y6eBhAbEiwA_2ZWIXM_kikDInXiiPvVBC3x2iTJupVnh2KJ_dtFgHGbGuo0cINq0VxykBoCUEkQAvD_BwE'
driver=webdriver.Chrome()
driver.get(url)

In [140]:
driver.find_element(By.ID, value='AKCSearch').send_keys('무선청소기')
driver.find_element(By.CSS_SELECTOR, value='.search__submit').click()

In [141]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('ul.product_list > li.prod_item')
len(lis)

44

### 1. 다나와 데이터 수집(1페이지 크롤링)

In [142]:
li=lis[0]
li

<li class="prod_item" id="productItem15170318">
<input id="productItem_categoryInfo_15170318" type="hidden" value="생활가전_청소기"/>
<input id="min_price_15170318" type="hidden" value="458920"/>
<div class="prod_main_info">
<div class="thumb_image">
<a class="thumb_link click_log_product_standard_img_" href="https://prod.danawa.com/info/?pcode=15170318&amp;keyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&amp;cate=10325109" id="thumbLink_15170318" onmousedown="_trkEventLog('통합검색_상품블로그_유입');" target="_blank">
<img alt="제트 VS20A956A3" onerror="this.onerror=null; this.src='//img.danawa.com/new/noData/img/noImg_160.gif';" src="//img.danawa.com/prod_img/500000/318/170/img/15170318_1.jpg?shrink=130:130&amp;_v=20210906145334"/>
</a>
<a class="btn_view_zoom" data-assemblygallerycategory="N" href="#" id="imageZoom_15170318" onclick="return false;">
<span class="ico i_zoom_vw"></span>이미지보기
										</a>
<input id="hidden_cate_sub_c1" type="hidden" value="72"/>
<input id="hidden_cate_sub_c2" type=

In [143]:
name=li.select_one('p.prod_name > a').get_text().strip()
name

'삼성전자 비스포크 제트 VS20A956A3'

In [146]:
info=li.select_one('div.spec_list').get_text().replace('\t','').replace('\n','')
info

'핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 210W / [구성] 바닥 / 솔형 / 틈새 / 연장툴 / 청정스테이션 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간(개당): 1시간 / [성능] 디지털인버터모터 / 5단계여과 / [기능] 물걸레(별도구매) / 디스플레이표시창 / 자동물공급 / 배터리잔량표시 / 자동먼지비움 / [부가] 먼지통용량: 0.5L / 물통용량: 0.15L / 색상: 미드나잇블루 / 무게: 2.5kg / 액세서리크래들  / 크기(가로x세로x깊이): 250x930x202mm'

In [147]:
price=li.select_one('p.price_sect > a > strong').text.replace(',','')
price

'458920'

In [156]:
# 1페이지에 대한 상품정보 추출
prod_data = []
for li in lis:
    # 공백 ('li.prod_itme.product-pot') 제외
    if 'product-pot' in li['class']:
        continue
    try:
        name = li.select_one('p.prod_name > a').text.strip()
        info = li.select_one('div.spec_list').text.replace('\t', '').replace('\n', '')
        price = li.select_one('p.price_sect > a > strong').text.strip().replace(',', '')
        prod_data.append([name, info, price])
    except:
        pass
print(len(prod_data))
print(prod_data)

40
[['삼성전자 비스포크 제트 VS20A956A3', '핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 210W / [구성] 바닥 / 솔형 / 틈새 / 연장툴 / 청정스테이션 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간(개당): 1시간 / [성능] 디지털인버터모터 / 5단계여과 / [기능] 물걸레(별도구매) / 디스플레이표시창 / 자동물공급 / 배터리잔량표시 / 자동먼지비움 / [부가] 먼지통용량: 0.5L / 물통용량: 0.15L / 색상: 미드나잇블루 / 무게: 2.5kg / 액세서리크래들  / 크기(가로x세로x깊이): 250x930x202mm', '458920'], ['베이스어스 차량용 청소기 A3 (해외구매)', '차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm', '48360'], ['LG전자 오브제컬렉션 코드제로 A9S AU9982', '핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 와이드바닥 / 스팀물걸레 / 침구 / 솔형 / 틈새 / 올인원타워 / [배터리] 분리형 (2개) / 충전시간: 4시간 / 사용시간(개당): 1시간 / [성능] 스마트인버터모터 / 헤파필터 / 5단계여과 / [기능] IoT기능 / 디스플레이표시창 / 자동물공급 / LED라이트 / 자동먼지비움 / [부가] 색상: 카밍베이지 / 무게: 2.7kg / 크기(가로x세로x깊이): 300x1120x260mm', '1047800'], ['삼성전자 비스포크 제트 VS20B956AX', '핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 220W / [구성] 바닥 / 솔형 / 틈새 / 연장툴 / 청정스테이션 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간(개

In [161]:
# 성품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for li in lis:
        # 공백 ('li.prod_itme.product-pot') 제외
        if 'product-pot' in li['class']:
            continue
        try:
            name = li.select_one('p.prod_name > a').text.strip()
            info = li.select_one('div.spec_list').text.replace('\t', '').replace('\n', '')
            price = li.select_one('li.rank_one > p.price_sect > a > strong').text.strip().replace(',', '')
            prod_data.append([name, info, price])
        except:
            pass
    return prod_data

In [162]:
# 상품 정보를 가져오는 함수 테스트
prod_items=soup.select('div.main_prodlist > ul.product_list > li.prod_item')
prod_data=get_prod_items(prod_items)
print(len(prod_data))

40


### 2. 여러페이지에 걸친 다나와 검색 페이지 크롤링

In [163]:
# 다나와 검색 URL을 만들어주는 함수
def get_search_page_url(keyword, page):
    return f'https://search.danawa.com/dsearch.php?query={keyword}&originalQuery={keyword}&previousKeyword={keyword}&volumeType=allvs&page={page}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=2942&defaultVaTab=719648&tab=main'

keyword='무선청소기'
page=1
url=get_search_page_url(keyword, page)
print(url)

https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&volumeType=allvs&page=1&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=2942&defaultVaTab=719648&tab=main


In [164]:
from tqdm.notebook import tqdm

In [165]:
keyword='무선청소기'
total_page=10
prod_data_total=[]
for page in tqdm(range(1, total_page+1)):
    url=get_search_page_url(keyword, page)
    driver.get(url)
    time.sleep(5)
    
    html=driver.page_source
    soup=BeautifulSoup(html, 'html.parser')
    
    #상품정보 추출
    prod_items=soup.select('div#productListArea > div.main_prodlist > ul.product > li.prod_item')
    prod_item_list=get_prod_items(prod_items)
    
    #추출한 데이터 저장
    prod_data_total=prod_data_total+prod_item_list

  0%|          | 0/10 [00:00<?, ?it/s]

In [166]:
len(prod_data_total)

400

### 3. 수집한 데이터 저장

In [167]:
import pandas as pd
df=pd.DataFrame(prod_data_total)
df.columns=['상품명','스펙 목록','가격']
df.to_csv('danawa_crawling_result.csv', index=False)